# Assignment 2: The Winter is here
##### This works best with epic battle music. No spoilers present.
<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;Tywin Lannister was right when he said: "The great war is between death and life, ice and fire. If we loose, the night will never end"<br/>
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;It has been six months since the white walkers' army marched into the north, led by the night king himself on a dead dragon. It has been a battle like never before: never before have men faced such an enemy in battle, never before have men fought so bravely against a united threat, and never before have they been so gravely defeated.<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; While Cersei is in King's landing, brave men have died fighting the great war. Among others, Tyrion is dead, Arya is dead and Jon Snow is dead, again. In a desperate battle, Daenerys leads all her forces in a final stand-off with the dead just south of Winterfell. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Her army defeated, she is now on the run on her dragon in an air battle, being chased by two of her own dragons, the Night king and a dead Jon Snow. Suddenly, the Night king's spear hits Danny's dragon, who, raining blood and fire, falls into ice, taking the lost queen, with him. <br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Daenerys opens her eyes in a strange place, a place which does not follow the rules of space and time, where the dead souls killed by the dead men are trapped, forever. But who woke her up? There stands near her, Tyrion, with Jorah, Davos, Jon Snow, and everybody else. They all indulge in a heartfelt reunion when someone yells- "But how do we get out?<br />
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; Varys sees a talking crystal close by, who asks them of completing a task, which on completion would allow them to go back to the land of the living, with the ultimate tool to defeat the white-walkers and kills the night king, the Dragon-axe. They have summoned you for help, as the task is out of their expertise, to apply a modified CNN to solve the object detection problem on the PASCAL VOC dataset. Varys, the master of whisperers, has used his talents to import the following for you:

In [ ]:
from __future__ import division, print_function, unicode_literals
import numpy as np
import torch
import torch.utils.data
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
import xml.etree.ElementTree as ET
import os
from PIL import Image


%matplotlib inline
plt.ion()
# You can ask Varys to get you more if you desire

In [107]:
import random
from collections import namedtuple

In [2]:
resnet_input = 224 #size of resnet18 input images

In [3]:
# Cersei chose violence, you choose your hyper-parameters wisely using validation data!
batch_size = 2
num_epochs = 5
learning_rate =  0.001
hyp_momentum = 0.9

## Build the data
The hound who was in charge for getting the data, brought you the following links:
<br/>Training and validation:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtrainval_06-Nov-2007.tar
<br/>Testing data:
<br/>http://host.robots.ox.ac.uk/pascal/VOC/voc2007/VOCtest_06-Nov-2007.tar
<br/>He also told you that the dataset(datascrolls :P) consists of images from of 20 classes, with detection annotations included. The JPEGImages folder houses the images, and the Annotations folder has the object-wise labels for the objects in one xml file per image. You have to extract the object information, ie. the [xmin, ymin] (the top left x,y co-ordinates) and the [xmax, ymax] (the bottom right x,y co-ordinates) of only the objects belonging to the given 20 classes(aeroplane, bicycle, boat, bottle, bus, car, cat, chair, cow, dining table, dog, horse, motorbike, person, potted plant, sheep, train, TV). For parsing the xml file, you can ask Varys to import xml.etree.ElementTree for you. <br/>
<br/> You can then ask Bronn and Jamie to organize the data as follows:
<br/> For every image in the dataset, extract/crop the object patch from the image one by one using their respective co-ordinates:[xmin, ymin, xmax, ymax], resize the image to resnet_input, and store it with its class label information. Do the same for training/validation and test datasets. <br/>
##### Important
You also have to collect data for an extra background class which stands for the class of an object which is not a part of any of the 20 classes. For this, you can crop and resize any random patches from an image. A good idea is to extract patches that have low "intersection over union" with any object present in the image frame from the 20 Pascal VOC classes. The number of background images should be roughly around those of other class objects' images. Hence the total classes turn out to be 21. This is important for applying the sliding window method later.

In [4]:
classes = ('__background__',
           'aeroplane', 'bicycle', 'bird', 'boat',
           'bottle', 'bus', 'car', 'cat', 'chair',
           'cow', 'diningtable', 'dog', 'horse',
           'motorbike', 'person', 'pottedplant',
           'sheep', 'sofa', 'train', 'tvmonitor')
counts = [0]*21

In [119]:
Rectangle = namedtuple('Rectangle', 'xmin ymin xmax ymax')

def area(a, b):  # returns None if rectangles don't intersect
    dx = min(a.xmax, b.xmax) - max(a.xmin, b.xmin)
    dy = min(a.ymax, b.ymax) - max(a.ymin, b.ymin)
    if (dx>=0) and (dy>=0):
        return dx*dy
    
ra = Rectangle(1, 1, 3, 3)
rb = Rectangle(1, 1, 5, 5)

type(area(ra,rb))

int

def create_train_hierarchical_dirs():
    if not os.path.exists('data/train'):
        os.makedirs('data/train')
        
    for i in range(len(classes)):
        dirname = 'data/train/' + classes[i]
        if not os.path.exists(dirname):
            os.makedirs(dirname)

create_train_hierarchical_dirs()

def jamie_bronn_build_dataset():
    
    dir_names = os.listdir('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations')
    for img_name in dir_names:
        tree = ET.parse('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + img_name)
        root = tree.getroot()
        for object1 in root.iter('object'):
            for name1 in object1.iter('name'):
                dirname = name1.text
            for bndbox in object1.iter('bndbox'):
                for xmin in object1.iter('xmin'):
                    x1 = int(float(xmin.text))
                for xmax in object1.iter('xmax'):
                    x2 = int(float(xmax.text))
                for ymin in object1.iter('ymin'):
                    y1 = int(float(ymin.text))
                for ymax in object1.iter('ymax'):
                    y2 = int(float(ymax.text))

                img = Image.open('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/' + img_name[0:-4] + '.jpg')
                img = img.crop((x1,y1,x2,y2))
                if dirname in classes:
                    img.save('data/train/' + dirname + '/' + str(counts[classes.index(dirname)]) + ".jpg");
                    counts[classes.index(dirname)]+=1
        
jamie_bronn_build_dataset()

In [168]:
def build_background_class():
    dir_names = os.listdir('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations')
    average = int(sum(counts[1:])/len(counts[1:]))
    counts[0] = 0
    while(counts[0]!=average):        
        print(counts[0])
        select = 1
        img_name = random.choice(dir_names)
        tree = ET.parse('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/Annotations/' + img_name)
        root = tree.getroot()
        img = Image.open('data/VOCtrainval_06-Nov-2007/VOCdevkit/VOC2007/JPEGImages/' + img_name[0:-4] + '.jpg')
        imsize = img.size;
        x1 = random.randrange(imsize[0])
        x2 = random.randrange(imsize[0])
        y1 = random.randrange(imsize[1])
        y2 = random.randrange(imsize[1])
        if(x1>x2):
            temp = x1
            x1=x2
            x2=temp
        if(y1>y2):
            temp = y1
            y1=y2
            y2=temp
        print(x1,y1,x2,y2)
        if(x1==x2 or y1==y2):
            continue
        for object1 in root.iter('object'):
            for name1 in object1.iter('name'):
                dirname = name1.text
            for bndbox in object1.iter('bndbox'):
                intersection = 0
                for xmin in object1.iter('xmin'):
                    xa = int(float(xmin.text))
                for xmax in object1.iter('xmax'):
                    xb = int(float(xmax.text))
                for ymin in object1.iter('ymin'):
                    ya = int(float(ymin.text))
                for ymax in object1.iter('ymax'):
                    yb = int(float(ymax.text))
                ra = Rectangle(x1, x2, y1, y2)
                rb = Rectangle(xa, xb, ya, yb)
                if(area(ra,rb)!=None):
                    intersection = area(ra,rb)
                union = ((x2-x1)*(y2-y1))+((xb-xa)*(yb-ya))-intersection
                if(float(intersection)/union>0.5):
                    select = 0

        if(select==1):
            img = img.crop((x1,y1,x2,y2))
            img.save('data/train/__background__/' + str(counts[0]) + ".jpg");
            counts[0] += 1
    
build_background_class()

0
32 130 220 343
1
242 54 268 72
2
21 326 128 430
3
344 85 428 246
4
40 51 369 206
5
264 46 322 92
6
366 225 401 292
7
89 248 145 327
8
221 101 275 244
9
16 53 123 328
10
7 19 112 324
11
38 68 482 343
12
128 123 221 165
13
125 50 272 320
14
284 116 331 198
15
49 17 117 246
16
79 54 181 128
17
455 69 460 124
18
154 99 191 330
19
114 46 419 281
20
3 106 196 176
21
245 57 282 170
22
341 77 368 107
23
116 223 195 313
24
98 29 151 479
25
113 16 458 61
26
330 225 460 360
27
188 6 362 128
28
340 35 448 305
29
317 238 341 265
30
31 113 435 267
31
253 141 425 262
32
226 86 244 354
33
5 15 448 161
34
237 71 461 162
35
39 237 96 395
36
52 228 337 319
37
444 33 494 313
38
62 165 141 336
39
179 31 287 107
40
374 95 374 203
40
403 3 430 314
41
144 255 316 383
42
72 215 382 337
43
205 209 496 227
44
28 43 387 95
45
166 42 322 292
46
53 64 305 78
47
214 65 323 159
48
13 100 329 196
49
380 133 494 138
50
435 166 499 300
51
276 111 463 189
52
43 284 52 311
53
94 130 200 323
54
4 41 86 400
55
35 153 335 

438
90 105 94 133
439
146 187 214 322
440
233 124 256 259
441
3 148 374 309
442
77 190 348 267
443
255 89 476 347
444
122 137 337 357
445
255 126 272 393
446
298 175 456 253
447
78 78 400 105
448
102 153 456 374
449
134 149 358 368
450
122 52 187 135
451
318 15 451 85
452
167 93 230 103
453
85 156 269 366
454
195 172 493 218
455
214 61 249 464
456
116 249 460 370
457
94 215 426 226
458
104 344 308 346
459
262 109 266 390
460
180 9 350 190
461
271 7 357 156
462
281 187 320 250
463
234 114 366 199
464
36 364 156 365
465
85 311 285 419
466
306 251 347 266
467
341 38 371 140
468
358 64 416 140
469
340 198 453 301
470
257 254 343 309
471
309 102 444 170
472
58 38 479 305
473
230 5 386 128
474
392 252 499 346
475
110 278 286 439
476
408 131 468 289
477
9 163 478 321
478
125 81 126 372
479
85 52 260 224
480
37 35 320 240
481
262 184 417 230
482
173 75 475 226
483
138 70 168 327
484
29 86 466 193
485
158 88 317 247
486
120 201 454 225
487
54 153 192 190
488
104 292 346 486
489
212 239 471 304


In [ ]:
class hound_dataset(torch.utils.data.Dataset): # Extend PyTorch's Dataset class
    def __init__(self, root_dir, train, transform=None):
        # Begin
        
    def __len__(self):
        # Begin
        
    def __getitem__(self, idx):
       # Begin
    

## Train the netwok
<br/>You can ask Arya to train the network on the created dataset. This will yield a classification network on the 21 classes of the VOC dataset. 

In [ ]:
composed_transform = transforms.Compose([transforms.Scale((resnet_input,resnet_input)),
                                         transforms.ToTensor(),
                                         transforms.RandomHorizontalFlip()])
train_dataset = hound_dataset(root_dir='', train=True, transform=composed_transform) # Supply proper root_dir
test_dataset = hound_dataset(root_dir='', train=False, transform=composed_transform) # Supply proper root_dir

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

### Fine-tuning
Litlefinger has brought you a pre-trained network. Fine-tune the network in the following section:

In [ ]:
resnet18 = models.resnet18(pretrained=True)

resnet18.fc = nn.Linear(resnet18.fc.in_features, 21)

# Add code for using CUDA here

In [ ]:
criterion = nn.CrossEntropyLoss()
# Update if any errors occur
optimizer = optim.SGD(resnet18.parameters(), learning_rate, hyp_momentum)

In [ ]:
def arya_train():
    # Begin

In [ ]:
%time arya_train()

# Testing and Accuracy Calculation
Jorah then asks a question, how is this a detection task?<br/>
As everybody wonders, Theon Greyjoy suggests a slding window method to test the above trained trained network on the detection task:<br/>
"We take some windows of varying size and aspect ratios", he mumbled, "and slide it through the test image (considering some stride of pixels) from left to right, and top to bottom, detect the class scores for each of the window, and keep only those which are above a certain threshold value!". "He is right", says Samwell, "I read a similar approach in the paper -Faster RCNN by Ross Girshick in the library, where he uses three diferent scales/sizes and three different aspect ratios, making a total of nine windows per pixel to slide". You need to write the code and use it in testing code to find the predicted boxes and their classes.

In [ ]:
def theon_sliding_window():
    # Begin

"Wait", says <b>Jon Snow</b>, "The predicted boxes may be too many and we can't deal with all of them. So, I myself will go and apply non_maximum_supression to reduce the number of boxes". You are free to choose the threshold value for non maximum supression, but choose wisely [0,1].

In [ ]:
def aegon_targaryen_non_maximum_supression(boxes,threshold = 0.3):
    # 

Daenerys, the queen, then orders her army to test out the trained model on the test dataset.

In [ ]:
def daenerys_test(resnet18):
    # Write loops for testing the model on the test set
    # Also print out the accuracy of the model

In [ ]:
%time daenerys_test(resnet18)

# Final Showdown
After covering all the steps and passing the accuracy value to the talking crystal, they all pass through to the land of the living, with a wounded Jon Snow armed with the Dragon-axe. After a fierce battle, Jon Snow manages to go face to face with the Night king. Surrounded by battling men and falling bodies, they engage in a ferocious battle, a battle of spear and axe. After a raging fight, Jon manages to sink the axe into the Night king's heart, but not before he gets wounded by the spear. As dead men fall to bones, Daenerys and others rush to his aid, but it is too late. Everyone is in tears as they look towards the man of honour, Jon Snow, lying in Daenerys's arms when he says his last words: "The night has ended. Winter is finally over!"